In [1]:
# paraPropPython time-dependent signal example use of paraPropPython.py notebook
# s. prohira, c. sbrocco
import sys
sys.path.append('../')
%matplotlib inline
import paraPropPython as ppp
from receiver import receiver
import numpy as np
import matplotlib.pyplot as plt
import util as util
import time

In [2]:
#### time-dependent example #####

### first, initialize an instance of paraProp by defining its dimensions and frequency of interest ###
iceDepth = 1000. # m
iceLength = 110. # m
dx = 1 # m
dz = 0.01 # m

freq = 0.2

### it is useful to set the reference depth as the source depth when you only have one transmitter ###
sourceDepth = 10. # m
sim = ppp.paraProp(iceLength, iceDepth, dx, dz, refDepth=1, airHeight=1)

### useful arrays for plottinng ###
z = sim.get_z()
x = sim.get_x()

In [3]:
### NOTE: define n before defining source ###
def get_profile_from_file(fname):
    profile_data = np.genfromtxt(fname)
    z_profile = profile_data[:,0]
    n_profile = profile_data[:,1]
    return n_profile, z_profile

### an example of defining n as a function of z (also can be done using a vector, see implementation) ###
rho_vec0, z_vec0 = get_profile_from_file('NP_data_dzb.txt')
rho_vec0 = rho_vec0/1e3

A0 = 1.0
B0 = 0.841
def rho2refindex(rho, A, B):
    return A + B * rho

z_vec = np.arange(0, iceDepth, dz)
rho_vec = np.interp(z_vec, z_vec0, rho_vec0)

n_vec = rho2refindex(rho_vec, A0, B0)
sim.set_n(nVec=n_vec, zVec=z_vec)

In [4]:
#Simulation Geometry
iceDepth = 60. # m
iceLength = 110. # m
dx = 1 # m
dz = 0.05 # m

#Frequency
freq = 0.2

#Tx and Rx
sourceDepth0 = 10. # m
sourceDepth1 = 20. # m
sourceDepth2 = 50. # m
receiverL0 = 30.
receiverL1 = 100.
receiverDepth0 = 10.
receiverDepth1 = 20.
receiverDepth2 = 50.
receiverRange = [receiverL0, receiverL1]
sourceDepth= [sourceDepth0,sourceDepth1,sourceDepth2]
receiverDepth = [receiverDepth0,receiverDepth1,receiverDepth2]
#Signal Properties
### set a td signal ###

#Refractive Index
### NOTE: define n before defining source ###
def get_profile_from_file(fname):
    profile_data = np.genfromtxt(fname)
    z_profile = profile_data[:,0]
    n_profile = profile_data[:,1]
    return n_profile, z_profile


def get_rx_pulse(z_vec, n_vec, z_tx=sourceDepth, x_rx=receiverRange, z_rx=receiverDepth):
    dt = 1
    impulse = np.zeros(2**8, dtype='complex')
    impulse[10] = 1+0j
    sig = util.normToMax(util.butterBandpassFilter(impulse, 0.09, 0.25, 1/dt, 4))

    sim = ppp.paraProp(iceLength, iceDepth, dx, dz, refDepth=1, airHeight=1)
    sim.set_n(nVec=n_vec, zVec=z_vec)
    sim.set_dipole_source_profile(freq, z_tx)
    sim.set_td_source_signal(sig, dt)
    rxList = [receiver(x_rx, z_rx)]
    ### run the solver ###
    sim.do_solver(rxList)
    rx = rxList[0]
    t = rx.get_time()
    sig_rx = rx.get_signal().real
    f = rx.get_frequency()
    spec_rx = rx.get_spectrum()
    return sig_rx, t, spec_rx, f
def rho2refindex(rho, A, B):
    return A + B * rho

In [6]:
import numpy as np

# Define the functions to compute refractive indices
def cubedn(rho, A, B):
    return np.sqrt((A + B * rho) ** 3)

def rho2refindex(rho, A, B):
    return A + B * rho

# Parameters
A1, A2, A3 = 1.0, 0.988, 0.992
B1, B2, B3, B4, B5 = 0.840, 0.845, 0.850, 0.848, 0.508

# Assume these functions are defined to get data from a file
rho_vec0, z_vec0 = get_profile_from_file('NP_data_dzb.txt')
rho_vec0 = rho_vec0 / 1e3  # Convert units if necessary

# Define ice depth and step size
iceDepth = 1000  # Maximum depth in meters
dz = .01          # Depth step size in meters

# Generate depth values
z_vec = np.arange(0, iceDepth, dz)
rho_vec = np.interp(z_vec, z_vec0, rho_vec0)  # Interpolate density profile

# Generate refractive indices for each model
n_vec_list = [
    rho2refindex(rho_vec, A1, B1),
    rho2refindex(rho_vec, A1, B2),
    rho2refindex(rho_vec, A2, B2),
    rho2refindex(rho_vec, A3, B4),
    rho2refindex(rho_vec, A2, B3),
    cubedn(rho_vec, A1, B5)
]
B_list = [B1, B2, B2, B4, B3, B5]
A_list = [A1, A1, A2, A3, A2, A1]

# Create separate text files for each refractive index model
for i, n_vec in enumerate(n_vec_list):
    filename = f"refractive_index_model_{i + 1}.txt"
    with open(filename, "w") as f:
        np.savetxt(f, np.column_stack((z_vec, n_vec)), fmt="%.4f")

print("Text files generated successfully.")


Text files generated successfully.
